

## Load our dataset :

In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.1.0+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv,GATConv,GMMConv
from torch_geometric.utils import train_test_split_edges
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn.models import InnerProductDecoder, VGAE
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops
import torch.nn as nn
from torch.optim import Adam
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges


1. adjacency martix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
weighted_adjacency_matrix = np.load('/content/drive/My Drive/Data/ProcessedData/World/adjacency matrix/5400/reduced_graph_a=1.5_b=0.5_sp=0.044.npy')

In [ ]:
# weighted_adjacency_matrix[weighted_adjacency_matrix < 0.7] = 0

In [ ]:
edge_index = np.array(weighted_adjacency_matrix.nonzero())
np.shape(edge_index)

(2, 685176)

2. feature matrix

In [ ]:
feature_matrix = np.load('/content/drive/My Drive/Data/ProcessedData/World/features/final feature mat/reduced_feature_mat4.npy')
feature_matrix = np.nan_to_num(feature_matrix)

In [ ]:
from torch_geometric.data import Data

In [ ]:
# Convert the adjacency matrix to a COO format
edge_index = np.array(weighted_adjacency_matrix.nonzero())#.t()
np.shape(edge_index)

(2, 685176)

In [ ]:

# Convert the feature matrix to PyTorch tensor
x = torch.tensor(feature_matrix, dtype=torch.float)

# Create the PyG Data object
data = Data(x=x, edge_index=edge_index)#,edge_weight=np.random.randn(100))

# Print the created Data object
print(data)

Data(x=[3294, 484], edge_index=[2, 685176])


In [ ]:
temp = data.edge_index
temp = np.transpose(temp)

In [ ]:
edges = []
for i,j in temp:
    if i != j:
        edges.append((i,j))

In [ ]:
edges = np.transpose(edges)
edges.shape

(2, 685176)

In [ ]:
weights = []
for i,j in np.transpose(data.edge_index):
    # print(i,j)
    if i!=j:
        weights.append(weighted_adjacency_matrix[i,j])


In [ ]:
weights = np.array(weights)
weights.shape, weights[0]

((685176,), 0.04350505946947198)

In [ ]:
# Convert the feature matrix to PyTorch tensor
x = torch.tensor(feature_matrix, dtype=torch.float)

# Create the PyG Data object
data = Data(x=x, edge_index=torch.tensor(edges),edge_attr=torch.tensor(weights).unsqueeze(1)) # edge_weight

# Print the created Data object
print(data)

Data(x=[3294, 484], edge_index=[2, 685176], edge_attr=[685176, 1])


----
----
----
# VGAE

In [ ]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.gcn_shared1 = GATConv(in_channels, hidden_channels)
        self.gcn_shared2 = GATConv(hidden_channels, hidden_channels)
        self.gcn_shared3 = GATConv(hidden_channels*2, hidden_channels)

        self.gcn_mu = GATConv(hidden_channels, out_channels)
        self.gcn_logvar = GATConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.gcn_shared1(x, edge_index))
        # print('q1 ', x.shape)
        x1 = F.relu(self.gcn_shared2(x, edge_index))
        # print('q2 ', x1.shape)
        # print('q3 ',torch.concat((x1,x),dim=1).shape)
        x2 = F.relu(self.gcn_shared3(torch.concat((x1,x),dim=1), edge_index)) # Skip connection

        mu = self.gcn_mu(x2, edge_index)
        logvar = self.gcn_logvar(x2, edge_index)
        return mu, logvar


class DeepVGAE(VGAE):
    def __init__(self, num_features, out_channels):
        super(DeepVGAE, self).__init__(encoder=GCNEncoder(num_features,128, out_channels),
                                       decoder=InnerProductDecoder())

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        adj_pred = self.decoder.forward_all(z)
        return adj_pred

    def loss(self, x, pos_edge_index, all_edge_index):
        z = self.encode(x, pos_edge_index)

        pos_loss = -torch.log(
            self.decoder(z, pos_edge_index, sigmoid=True) + 1e-15).mean()

        # Do not include self-loops in negative samples
        all_edge_index_tmp, _ = remove_self_loops(all_edge_index)
        all_edge_index_tmp, _ = add_self_loops(all_edge_index_tmp)

        neg_edge_index = negative_sampling(all_edge_index_tmp, z.size(0), pos_edge_index.size(1))
        neg_loss = -torch.log(1 - self.decoder(z, neg_edge_index, sigmoid=True) + 1e-15).mean()

        kl_loss = 1 / x.size(0) * self.kl_loss()

        return pos_loss + neg_loss + kl_loss

    def single_test(self, x, train_pos_edge_index, test_pos_edge_index, test_neg_edge_index):
        with torch.no_grad():
            z = self.encode(x, train_pos_edge_index)
        roc_auc_score, average_precision_score = self.test(z, test_pos_edge_index, test_neg_edge_index)
        return roc_auc_score, average_precision_score

In [ ]:
# torch.manual_seed(12345)
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# parameters
out_channels = 64
num_features = data.num_features
epochs = 200

model = DeepVGAE(num_features, out_channels).to(device)
optimizer = Adam(model.parameters(), lr=0.01)

data = data.to(device)
all_edge_index = data.edge_index
# print('wwww ', data.edge_attr)

data = train_test_split_edges(data, 0.05, 0.1)

# print('wwww ', data.train_pos_edge_index.edge_attr)

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    loss = model.loss(data.x, data.train_pos_edge_index, all_edge_index)
    loss.backward()
    optimizer.step()
    if epoch % 2 == 0:
        model.eval()
        roc_auc, ap = model.single_test(data.x,
                                        data.train_pos_edge_index,
                                        data.test_pos_edge_index,
                                        data.test_neg_edge_index)
        print("Epoch {} - Loss: {:.3f} ROC_AUC: {:.3f} Precision: {:.3f}".format(epoch, loss.cpu().item(), roc_auc, ap))

<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Epoch 0 - Loss: 23553.123 ROC_AUC: 0.500 Precision: 0.500
Epoch 2 - Loss: 74025.164 ROC_AUC: 0.500 Precision: 0.500
Epoch 4 - Loss: 21051.508 ROC_AUC: 0.500 Precision: 0.500
Epoch 6 - Loss: 35.625 ROC_AUC: 0.500 Precision: 0.500
Epoch 8 - Loss: 38.149 ROC_AUC: 0.500 Precision: 0.500
Epoch 10 - Loss: 35.937 ROC_AUC: 0.500 Precision: 0.500
Epoch 12 - Loss: 35.991 ROC_AUC: 0.500 Precision: 0.500
Epoch 14 - Loss: 35.821 ROC_AUC: 0.500 Precision: 0.500
Epoch 16 - Loss: 35.217 ROC_AUC: 0.500 Precision: 0.500
Epoch 18 - Loss: 35.094 ROC_AUC: 0.500 Precision: 0.500
Epoch 20 - Loss: 35.149 ROC_AUC: 0.500 Precision: 0.500
Epoch 22 - Loss: 35.600 ROC_AUC: 0.500 Precision: 0.500
Epoch 24 - Loss: 36.072 ROC_AUC: 0.500 Precision: 0.500
Epoch 26 - Loss: 35.851 ROC_AUC: 0.500 Precision: 0.500
Epoch 28 - Loss: 35.447 ROC_AUC: 0.500 Precision: 0.500
Epoch 30 - Loss: 35.186 ROC_AUC: 0.500 Precision: 0.500
Epoch 32 - Loss: 35.164 ROC_AUC: 0.500 Precision: 0.500
Epoch 34 - Loss: 35.274 ROC_AUC: 0.500 Preci

In [ ]:
Z = model.encode(data.x, data.train_pos_edge_index)
Z.shape

torch.Size([3294, 64])

In [ ]:
torch.save(Z, '/content/drive/My Drive/Data/Node Embeddings/node_embedding18.pt')